In [538]:
import numpy as np 
import pandas as pd 

In [539]:
games = pd.read_csv('games.csv')
games.shape

(1512, 14)

In [540]:
games.head(1)

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K


In [541]:
#team
#rating
#genres
#title
#summary
#reviews

games = games[['Title','Team','Genres','Rating','Summary','Reviews']]

In [542]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Title    1512 non-null   object 
 1   Team     1511 non-null   object 
 2   Genres   1512 non-null   object 
 3   Rating   1499 non-null   float64
 4   Summary  1511 non-null   object 
 5   Reviews  1512 non-null   object 
dtypes: float64(1), object(5)
memory usage: 71.0+ KB


In [543]:
games.head(1512)

,Title,Team,Genres,Rating,Summary,Reviews
0,Elden Ring,"['Bandai Namco Entertainment', 'FromSoftware']","['Adventure', 'RPG']",4.5,"Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o..."
1,Hades,['Supergiant Games'],"['Adventure', 'Brawler', 'Indie', 'RPG']",4.3,A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...
2,The Legend of Zelda: Breath of the Wild,"['Nintendo', 'Nintendo EPD Production Group No...","['Adventure', 'RPG']",4.4,The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...
3,Undertale,"['tobyfox', '8-4']","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",4.2,"A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...
4,Hollow Knight,['Team Cherry'],"['Adventure', 'Indie', 'Platform']",4.4,A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with..."
...,...,...,...,...,...,...
1507,Back to the Future: The Game,['Telltale Games'],"['Adventure', 'Point-and-Click']",3.2,Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...
1508,Team Sonic Racing,"['Sumo Digital', 'Sega']","['Arcade', 'Racing']",2.9,Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C..."
1509,Dragon's Dogma,['Capcom'],"['Brawler', 'RPG']",3.7,"Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ..."
1510,Baldur's Gate 3,['Larian Studios'],"['Adventure', 'RPG', 'Strategy', 'Tactical', '...",4.1,"An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...


In [544]:
games.isnull().sum()

Title       0
Team        1
Genres      0
Rating     13
Summary     1
Reviews     0
dtype: int64

In [545]:
games.dropna(inplace=True)

In [546]:
games.duplicated().sum()

np.int64(384)

In [547]:
games=games.drop_duplicates()

In [548]:
#games['Tags']=games['Summary']+games['Reviews']+games['Genres']+games['Team']
games['Tags']=games['Summary']+games['Genres']+games['Team']
#seems like its giving better suggestion after removing the reviews

In [549]:
games.head()

,Title,Team,Genres,Rating,Summary,Reviews,Tags
0,Elden Ring,"['Bandai Namco Entertainment', 'FromSoftware']","['Adventure', 'RPG']",4.5,"Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...","Elden Ring is a fantasy, action and open world..."
1,Hades,['Supergiant Games'],"['Adventure', 'Brawler', 'Indie', 'RPG']",4.3,A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,A rogue-lite hack and slash dungeon crawler in...
2,The Legend of Zelda: Breath of the Wild,"['Nintendo', 'Nintendo EPD Production Group No...","['Adventure', 'RPG']",4.4,The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,The Legend of Zelda: Breath of the Wild is the...
3,Undertale,"['tobyfox', '8-4']","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",4.2,"A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,"A small child falls into the Underground, wher..."
4,Hollow Knight,['Team Cherry'],"['Adventure', 'Indie', 'Platform']",4.4,A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",A 2D metroidvania with an emphasis on close co...


In [550]:
new_df=games[['Title','Rating','Tags']]

In [551]:
new_df['Tags']=new_df['Tags'].apply(lambda x:x.lower())

/var/folders/9z/ckbkpd7x3r100l5l3b_bq7xr0000gn/T/ipykernel_31999/2750551276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Tags']=new_df['Tags'].apply(lambda x:x.lower())


In [552]:
new_df.head()

,Title,Rating,Tags
0,Elden Ring,4.5,"elden ring is a fantasy, action and open world..."
1,Hades,4.3,a rogue-lite hack and slash dungeon crawler in...
2,The Legend of Zelda: Breath of the Wild,4.4,the legend of zelda: breath of the wild is the...
3,Undertale,4.2,"a small child falls into the underground, wher..."
4,Hollow Knight,4.4,a 2d metroidvania with an emphasis on close co...


In [553]:
new_df['Tags'].info

<bound method Series.info of 0       elden ring is a fantasy, action and open world...
1       a rogue-lite hack and slash dungeon crawler in...
2       the legend of zelda: breath of the wild is the...
3       a small child falls into the underground, wher...
4       a 2d metroidvania with an emphasis on close co...
                              ...                        
1507    back to the future: the game is one of telltal...
1508    team sonic racing combines the best elements o...
1509    set in a huge open world, dragon’s dogma: dark...
1510    an ancient evil has returned to baldur's gate,...
1511    join emmet and an unlikely group of resistance...
Name: Tags, Length: 1114, dtype: object>

In [554]:
import re

clean_tag=""
counter=0
for i in new_df['Tags']:
   clean_tag = re.sub(r"['\"\\.\\\]\[]", " ", i)
   new_df['Tags'][counter]=clean_tag
   counter+=1


/var/folders/9z/ckbkpd7x3r100l5l3b_bq7xr0000gn/T/ipykernel_31999/4031792632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Tags'][counter]=clean_tag
/var/folders/9z/ckbkpd7x3r100l5l3b_bq7xr0000gn/T/ipykernel_31999/4031792632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Tags'][counter]=clean_tag
/var/folders/9z/ckbkpd7x3r100l5l3b_bq7xr0000gn/T/ipykernel_31999/4031792632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [555]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1114 entries, 0 to 1511
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   1114 non-null   object 
 1   Rating  1114 non-null   float64
 2   Tags    1114 non-null   object 
dtypes: float64(1), object(2)
memory usage: 67.1+ KB


In [556]:
import nltk

In [557]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [558]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y) 

In [559]:
new_df['Tags']=new_df['Tags'].apply(stem)

/var/folders/9z/ckbkpd7x3r100l5l3b_bq7xr0000gn/T/ipykernel_31999/793828252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Tags']=new_df['Tags'].apply(stem)


In [560]:
from sklearn.feature_extraction. text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words='english')

In [561]:
vectors = cv.fit_transform(new_df['Tags']).toarray()

In [562]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(8044,))

In [563]:
cv.get_feature_names_out()

array(['00', '000', '07th', ..., 'zoom', 'zx', 'âˆ'],
      shape=(8044,), dtype=object)

In [564]:
from sklearn.metrics.pairwise import cosine_similarity

In [565]:
similarity = cosine_similarity(vectors)

In [566]:
similarity

array([[1.        , 0.1       , 0.19364917, ..., 0.13421887, 0.06201737,
        0.02411214],
       [0.1       , 1.        , 0.12909944, ..., 0.09587062, 0.03100868,
        0.        ],
       [0.19364917, 0.12909944, 1.        , ..., 0.13614529, 0.04003204,
        0.09338592],
       ...,
       [0.13421887, 0.09587062, 0.13614529, ..., 1.        , 0.04756515,
        0.        ],
       [0.06201737, 0.03100868, 0.04003204, ..., 0.04756515, 1.        ,
        0.08972229],
       [0.02411214, 0.        , 0.09338592, ..., 0.        , 0.08972229,
        1.        ]], shape=(1114, 1114))

In [567]:
def recommend(game):
    game_index = new_df[new_df['Title']== game].index[0]
    distances = similarity[game_index]
    game_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x:x[1])[1:6]   

    for i in game_list:
        print(new_df.iloc[i[0]].Title)

In [589]:
recommend("Prototype")

Killzone 3
Grand Theft Auto V
Demon's Souls
Odin Sphere: Leifthrasir
Yume Nikki
